In [62]:
import pandas as pd

from core.operation import *

In [100]:
current_date = date.today().strftime('%Y-%m-%d')

In [101]:
# Get Stock Data
ml_ret = pd.read_parquet(get_live_stock() / 'trade_stock_ml_ret.parquet.brotli')
ml_trend = pd.read_parquet(get_live_stock() / 'trade_stock_ml_trend.parquet.brotli')
port_iv = pd.read_parquet(get_live_stock() / 'trade_stock_port_iv.parquet.brotli')
port_id = pd.read_parquet(get_live_stock() / 'trade_stock_port_id.parquet.brotli')
port_ivm = pd.read_parquet(get_live_stock() / 'trade_stock_port_ivm.parquet.brotli')
trend_mls = pd.read_parquet(get_live_stock() / 'trade_stock_trend_mls.parquet.brotli')
mrev_etf = pd.read_parquet(get_live_stock() / 'trade_stock_mrev_etf.parquet.brotli')
mrev_mkt = pd.read_parquet(get_live_stock() / 'trade_stock_mrev_mkt.parquet.brotli')

# Load Live Price
permno_data = pd.read_parquet(get_live_price() / 'data_permno_live.parquet.brotli')
mrev_etf_hedge_data = pd.read_parquet(get_live_price() / 'data_mrev_etf_hedge_live.parquet.brotli')
mrev_mkt_hedge_data = pd.read_parquet(get_live_price() / 'data_mrev_mkt_hedge_live.parquet.brotli')
trend_mls_com_data = pd.read_parquet(get_live_price() / 'data_trend_mls_com_live.parquet.brotli')
trend_mls_bond_data = pd.read_parquet(get_live_price() / 'data_trend_mls_bond_live.parquet.brotli')
ml_trend_re_data = pd.read_parquet(get_live_price() / 'data_ml_trend_re_live.parquet.brotli')
ml_trend_bond_data = pd.read_parquet(get_live_price() / 'data_ml_trend_bond_live.parquet.brotli')

# Merge data by 'date', 'ticker', 'type'
stock_data = pd.concat([ml_ret, ml_trend, port_iv, port_id, port_ivm, trend_mls, mrev_etf, mrev_mkt], axis=0)
stock_data = stock_data.groupby(level=['date', 'ticker', 'type']).sum()
stock_data = stock_data.loc[stock_data.index.get_level_values('date') == current_date]

# Merge price data
permno_data = permno_data.reset_index().set_index(['ticker', 'date'])
price_data = pd.concat([permno_data, mrev_etf_hedge_data, mrev_mkt_hedge_data, trend_mls_bond_data, trend_mls_com_data, ml_trend_re_data, ml_trend_bond_data], axis=0)
price_data = price_data.loc[~price_data.index.duplicated(keep='last')]

In [103]:
stock_data.sum()

weight    1.0
dtype: float64

In [52]:
x = port_id.copy(deep=True)

In [61]:
price_data.loc[price_data['Close'].isnull()]

,,permno,Close
ticker,date,,
HSTO,2024-02-13,14046.0,NaN
USO,2024-02-13,NaN,NaN


In [91]:
# Get Stock Data
ml_ret = pd.read_parquet(get_live_stock() / 'trade_stock_ml_ret.parquet.brotli')
ml_trend = pd.read_parquet(get_live_stock() / 'trade_stock_ml_trend.parquet.brotli')

# Merge data by 'date', 'ticker', 'type'
stock_data = pd.concat([ml_ret, ml_trend], axis=0)
stock_data = stock_data.groupby(level=['date', 'ticker', 'type']).sum()
stock_data = stock_data.loc[stock_data.index.get_level_values('date') == current_date]

In [92]:
stock_data.loc[stock_data.index.get_level_values('ticker') == 'SNAP']

weight
date       ticker type         
2024-02-15 SNAP   long    0.004
                  short   0.005

In [93]:
# Assuming 'df' is your DataFrame # Convert 'type' into positive and negative weights 
stock_data['signed_weight'] = np.where(stock_data.index.get_level_values('type') == 'long', stock_data['weight'], -stock_data['weight']) 
# Sum signed weights by ticker and date 
net_weights = stock_data.groupby(['date', 'ticker'])['signed_weight'].sum()
# Determine the type based on the sign of the net weight 
net_weights = net_weights.reset_index() 
net_weights['type'] = np.where(net_weights['signed_weight'] > 0, 'long', 'short') 
# Assign absolute values to the weight column 
net_weights['weight'] = net_weights['signed_weight'].abs() 
del net_weights['signed_weight'] 
# Remove the signed_weight column as it's no longer needed 
# Re-create the MultiIndex with the correct 'type' 
stock_data = net_weights.set_index(['date', 'ticker', 'type'])

In [95]:
stock_data.loc[stock_data.index.get_level_values('ticker') == 'SNAP']

,,,weight
date,ticker,type,
2024-02-15,SNAP,short,0.001
